# Reduced Basis Methods with pyMOR

In [ ]:
import sys
if 'pyodide' in sys.modules:
    import piplite
    await piplite.install(['pymor', 'matplotlib', 'ipython'])

## Building the FOM

### The Thermal Block Problem

Solve:

\begin{align}
- \nabla \cdot [d(x, \mu) \nabla u(x, \mu)] &= f(x),  & x &\in \Omega,\\
                                  u(x, \mu) &= 0,     & x &\in \partial\Omega,
\end{align}

where

\begin{align}
d(x, \mu) &= \sum_{q=1}^Q \mathbb{1}_{\Omega_q}, \\
f(x)      &= 1.
\end{align}

satisfying $\overline{\Omega} = \overline{\dot{\bigcup}_{i=1}^{Q} \Omega_q}$.

### Setting up an analytical description of the thermal block problem

The thermal block problem already comes with pyMOR:

In [ ]:
!pip install 'ipywidgets<8'  # for Binder

In [ ]:
from pymor.basic import *
p = thermal_block_problem([2,2])

Our problem is parameterized:

In [ ]:
p.parameters

### Looking at the definition

We can easily look at the definition of `p` by printing its `repr`:

In [ ]:
p

### Building a discrete model

We use the builtin discretizer `discretize_stationary_cg` to compute a finite-element discretization of the problem:

In [ ]:
from pymor.basic import *
fom, data = discretize_stationary_cg(p, diameter=1/100)

`fom` is a `Model`. It has the same `Parameters` as `p`:

In [ ]:
fom.parameters

### Looking at the model

`fom` inherits its structure from `p`:

In [ ]:
fom

### Note

> Using an `analyticalproblem` and a `discretizer` is just one way
  to build the FOM.
  
> Everything that follows works the same for a FOM built using an external PDE solver.

### Solving the FOM

Remember the FOM's parameters:

In [ ]:
fom.parameters

To `solve` the FOM, we need to specify values for those parameters:

In [ ]:
U = fom.solve({'diffusion': [1., 0.01, 0.1, 1]})

`U` is a `VectorArray`, an ordered collection of vectors of the same dimension:

In [ ]:
U

> There is not the notion of a single vector in pyMOR! Don't try to get hold of one!

### Some words about VectorArrays

Each `VectorArray` has a length, giving you the number of vectors:

In [ ]:
len(U)

Its dimension gives you the *uniform* size of each vector in the array:

In [ ]:
U.dim

### Supported Operations:

|                  |                                                        |
| :-               | :-                                                     |
| `+`/`-`/`*`      | elementwise addition/subtraction/scalar multiplication |
| `inner`          | matrix of inner products between all vectors           |
| `pairwise_inner` | list of pairwise inner products                        |
| `lincomb`        | linear combination of the vectors in the array         |
| `scal`           | in-place scalar multiplication                         |
| `axpy`           | in-place BLAS axpy operation                           |
| `dofs`           | return some degrees of freedom as NumPy array          |
| `norm`           | list of norms                                          |
| `append`         | append vectors from another array                      |

### Playing a bit with VectorArrays

> All `VectorArrays` are created by their `VectorSpace`

In [ ]:
V = fom.solution_space.empty()

Let's accumulate some solutions:

In [ ]:
for mu in p.parameter_space.sample_randomly(10):
    V.append(fom.solve(mu))

In [ ]:
# your code here ...

### Indexing
We can index a `VectorArray` using numbers, sequences of numbers, or slices, e.g.:

In [ ]:
V_indexed = V[3:6]

Indexing **always** create a view on the original array:

In [ ]:
print(V_indexed.is_view)
V_indexed *= 0
V.norm()

### Looking at the solution

We can use the `visualize` method to plot the solution:

In [ ]:
fom.visualize(U)

### Looking at the solution

An array with multiple elements is visualized as a time-series:

In [ ]:
fom.visualize(V)

### Is the solution really a solution?

We compute the residual:

In [ ]:
mu = fom.parameters.parse([1., 0.01, 0.1, 1])
U = fom.solve(mu)
(fom.operator.apply(U, mu=mu) - fom.rhs.as_vector(mu)).norm()

> If you implement a `Model`, make sure that `solve` really returns solutions with zero residual!

### So how is `fom.rhs` defined?

Let's look at it:

In [ ]:
fom.rhs

What does `as_vector` do?

In [ ]:
from pymor.tools.formatsrc import print_source
print_source(fom.rhs.as_vector)

## Reducing the FOM

### Building an approximation space

As before, we compute some random solution **snapshots** of the FOM, which will
span our **reduced** approximation space:

In [ ]:
snapshots = fom.solution_space.empty()
for mu in p.parameter_space.sample_randomly(10):
    snapshots.append(fom.solve(mu))

It's a good idea, to orthonormalize the basis:

In [ ]:
basis = gram_schmidt(snapshots)

### Projecting the Model

In pyMOR, ROMs are built using a `Reductor`. Let's pick the most basic `Reductor`
available for a `StationaryModel`:

In [ ]:
reductor = StationaryRBReductor(fom, basis)

Every reductor has a `reduce` method, which builds the ROM:

In [ ]:
rom = reductor.reduce()

### Comparing ROM and FOM

In [ ]:
fom

In [ ]:
rom

### Solving the ROM

To solve the ROM, we just use `solve` again,

In [ ]:
mu = fom.parameters.parse([1., 0.01, 0.1, 1])
u_rom = rom.solve(mu)

to get the reduced coefficients:

In [ ]:
u_rom

A high-dimensional representation is obtained from the `reductor`:

In [ ]:
U_rom = reductor.reconstruct(u_rom)

### Computing the MOR error

Let's compute the error:

In [ ]:
U = fom.solve(mu)
ERR = U - U_rom
ERR.norm() / U.norm()

and look at it:

In [ ]:
fom.visualize(ERR)

### Certified Reduced Basis Method

Let's use use a more sophisticated `reductor` which assembles an efficient
upper bound for the MOR error:

In [ ]:
reductor = CoerciveRBReductor(
   fom,
   product=fom.h1_0_semi_product,
   coercivity_estimator=ExpressionParameterFunctional('min(diffusion)', fom.parameters)
)

and build a basis using a greedy search over the parameter space:

In [ ]:
training_set = p.parameter_space.sample_uniformly(4)
print(training_set[0])

In [ ]:
greedy_data = rb_greedy(fom, reductor, training_set, max_extensions=20)
print(greedy_data.keys())
rom = greedy_data['rom']

### Testing the ROM

Let's compute the error again:

In [ ]:
mu = p.parameter_space.sample_randomly()
U = fom.solve(mu)
u_rom = rom.solve(mu)
ERR = U - reductor.reconstruct(u_rom)
ERR.norm(fom.h1_0_semi_product)

and compare it with the estimated error:

In [ ]:
rom.estimate_error(mu)

### Is it faster?

Finally, we compute some timings:

In [ ]:
from time import perf_counter
mus = p.parameter_space.sample_randomly(10)
tic = perf_counter()
for mu in mus:
    fom.solve(mu)
t_fom = perf_counter() - tic
tic = perf_counter()
for mu in mus:
    rom.solve(mu)
t_rom = perf_counter() - tic
print(f'Speedup: {t_fom/t_rom}')

### Some things to try

- Plot the MOR error vs. the dimension of the reduced space.
 
- Plot the speedup vs. the dimension of the reduced space.

- Compute the maximum/minimum efficiency of the error estimator over the parameter space.

- Try different numbers of subdomains.